In [ ]:
# Se importa la librería cv2 que es la biblioteca de OpenCV en Python, 
# la cual se usa para procesamiento de imágenes y visión por computadora.
import cv2

# Aquí se carga un clasificador pre-entrenado para detectar cuerpos completos. 
# Este archivo XML (haarcascade_fullbody.xml) contiene los datos que permiten detectar los cuerpos humanos en una imagen.
clasificador_cascade = cv2.CascadeClassifier('./files/haarcascade_fullbody.xml')

# Este bloque verifica si el clasificador se cargó correctamente. 
# Si no es así, el programa muestra un mensaje de error y termina la ejecución.
if clasificador_cascade.empty():
    print('El clasificador no se ha cargado correctamente')
    exit()

# Aquí se abre la cámara. El parámetro 1 indica que se está usando la segunda cámara conectada (el índice de la 
# cámara puede variar según el sistema). Si fuera 0, se usaría la cámara predeterminada.
captura_video = cv2.VideoCapture(0)

# En este bucle se capturan los fotogramas de la cámara uno a uno mediante cap.read(). 
# ret es un valor booleano que indica si se pudo leer el fotograma correctamente, y frame es la imagen capturada.
while True:
    # Capturar el fotograma desde la cámara
    ret, frame = captura_video.read()

    if not ret:
        break

    # Convertir el fotograma a escala de grises (mejora la detección)
    # Para mejorar la eficiencia de la detección, se convierte el fotograma original de color a escala de grises. 
    # Esto reduce la cantidad de datos que necesita procesar el clasificador.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar cuerpos en el fotograma
#     Aquí se realiza la detección de cuerpos en el fotograma en escala de grises. La función detectMultiScale devuelve una lista de coordenadas de los cuerpos detectados. Los parámetros son:

# scaleFactor: Ajusta la escala de la imagen en cada paso de la detección (1.1 significa que la imagen se escala en un 10%).

# minNeighbors: Define cuántos rectángulos vecinos debe tener una detección para ser considerada válida. Un valor más alto reduce las falsas detecciones.

# minSize: El tamaño mínimo de los cuerpos a detectar (en píxeles).
    bodies = clasificador_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(136, 136))

    # Contar el número de personas detectadas
    # Se cuenta cuántos cuerpos fueron detectados, es decir, cuántos rectángulos (cuerpos) fueron encontrados en la imagen.
    num_personas = len(bodies)

    # Dibujar los rectángulos alrededor de las personas detectadas
    # Por cada cuerpo detectado, se dibuja un rectángulo en el fotograma original. 
    # Los valores (x, y) son las coordenadas del vértice superior izquierdo del rectángulo, y (w, h) son el ancho y la altura del rectángulo. 
    # El color del rectángulo es (255, 0, 0) (rojo) y el grosor del borde es de 2 píxeles.
    for (x, y, w, h) in bodies:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Mostrar el número de personas detectadas en el fotograma
    # Se superpone un texto en la imagen que indica cuántas personas han sido detectadas. El texto se muestra en la esquina superior izquierda.
    cv2.putText(frame, f'Personas: {num_personas}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Mostrar el fotograma con las detecciones
    # Muestra el fotograma con los rectángulos dibujados alrededor de las personas detectadas en una ventana con el título 'Detección de personas'.
    cv2.imshow('Detección de personas', frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos y cerrar las ventanas
captura_video.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

# Inicializar el detector HOG con el clasificador preentrenado para personas
clasificador_person = cv2.HOGDescriptor()
clasificador_person.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Abrir la cámara (usa 0 o 1 dependiendo de cuál cámara tengas disponible)
captura_video = cv2.VideoCapture(0)

if not captura_video.isOpened():
    print("No se pudo abrir la cámara")
    exit()

while True:
    ret, frame = captura_video.read()
    if not ret:
        break

    # Redimensionar el frame para mayor velocidad (opcional)
    frame = cv2.resize(frame, (640, 480))

    # Convertir a escala de grises (mejora la detección)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplicar suavizado y ecualización para mejorar contraste
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.equalizeHist(gray)

    # Detección de personas usando HOG
    boxes, weights = clasificador_person.detectMultiScale(gray, winStride=(8, 8), padding=(8, 8), scale=1.05)

    # Contar el número de personas detectadas
    num_personas = len(boxes)

    # Dibujar los rectángulos sobre las detecciones
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Mostrar el número de personas detectadas
    cv2.putText(frame, f'Personas: {num_personas}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Mostrar el resultado en una ventana
    cv2.imshow('Detección de personas (HOG + SVM)', frame)

    # Salir si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
captura_video.release()
cv2.destroyAllWindows()
